In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import typing as t
from fdua_competition.vectorstore import build_vectorstore
from fdua_competition.enums import Mode, VectorStoreOption
from fdua_competition.rag import ResearchAssistant, RAG
from fdua_competition.enums import ChatModelOption
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores.base import VectorStore
from pydantic import BaseModel, Field

In [3]:
# RAGクラスを継承して, 新しいRAGを作成する例

## レスポンスの型式を定義
class FactCheckerResponse(BaseModel):
    score: str = Field(description="スコア")
    reason: str = Field(description="理由")
    reference: list[str] = Field(description="[参照した文章]")


# RAGクラスを継承して新しいRAGクラスを定義
# prompt_template(), build_payload(), output_structure() メソッドをオーバーライドする
class FactChecker(RAG):
    def __init__(
        self, vectorstore: VectorStore, chat_model_option: ChatModelOption = ChatModelOption.AZURE, language: str = "japanese"
    ):
        super().__init__(vectorstore=vectorstore, chat_model_option=chat_model_option)
        self.language = language

    @property
    def prompt_template(self) -> ChatPromptTemplate:
        """プロンプトのテンプレート"""
        return ChatPromptTemplate.from_messages(
            [
                ("system", "情報の正確さを与えられたコンテキストのみ使ってを10点満点で評価して. {language}"),
                ("system", "コンテキスト:\n{context}"),
                ("user", "チェック対象:\n{llm_response}"),
            ]
        )

    def build_payload(self, query: str) -> dict[str, t.Any]:
        """プロンプトテンプレートをこの辞書で埋める"""
        return {"llm_response": query, "language": self.language}

    @property
    def output_structure(self) -> BaseModel:
        """出力型式を定義する"""
        return FactCheckerResponse


# [end: research_assistant]

In [4]:
vectorstore = build_vectorstore("one_assistant_test", Mode.TEST, VectorStoreOption.CHROMA)

[prepare_vectorstore] chroma: /fdua-competition/.fdua-competition/vectorstores/chroma
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/9.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/8.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/10.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/6.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/7.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/5.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/4.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-co

In [5]:
assistant = ResearchAssistant(vectorstore)

In [6]:
res = assistant.invoke("電通グループの創業は何年ですか？")
print(res)

query='電通グループの創業は何年ですか？' response='電通グループは1901年に創業されました' reason='創業年は文書内で明記されており1901年と記されています' organization_name='電通グループ' contexts=['株式会社日本広告社創業 光永星郎1901年']


In [7]:
fact_checker = FactChecker(vectorstore)

print("prompt_template:")
print(fact_checker.prompt_template)

query = f"{res.query}: {res.response}"
print(f"\nquery: {query}")

fact_checked_res = fact_checker.invoke(query)
print(f"\nres: {fact_checked_res}")

prompt_template:
input_variables=['context', 'language', 'llm_response'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='情報の正確さを与えられたコンテキストのみ使ってを10点満点で評価して. {language}'), additional_kwargs={}), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='コンテキスト:\n{context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['llm_response'], input_types={}, partial_variables={}, template='チェック対象:\n{llm_response}'), additional_kwargs={})]

query: 電通グループの創業は何年ですか？: 電通グループは1901年に創業されました

res: score='10' reason='提供された情報は正確で、1901年に電通グループが創業されたことが明記されています。' reference=['日本広告社創立は1901年に光永星郎によって行われた。']
